In [1]:
#import libraries

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pyarrow

In [2]:
#import data
df_1 = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
df_2 = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [3]:
df_1.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [4]:
df_2.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [5]:
df = pd.concat([df_1,df_2])
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [6]:
df_1.shape   #Number of records in Jan 2021 FHV data

(1154112, 7)

In [7]:
#Average duration in Jan 2021 FHV
df_1['duration'] = df_1.dropOff_datetime - df_1.pickup_datetime
df_1.duration = df_1.duration.apply(lambda td: td.total_seconds() / 60)

df_1['duration'].describe() 

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [8]:
#Average duration in Feb 2021 FHV
df_2['duration'] = df_2.dropOff_datetime - df_2.pickup_datetime
df_2.duration = df_2.duration.apply(lambda td: td.total_seconds() / 60)

df_2['duration'].describe() 

count    1.037692e+06
mean     2.070699e+01
std      1.611084e+02
min      1.666667e-02
25%      8.000000e+00
50%      1.410000e+01
75%      2.373333e+01
max      1.109190e+05
Name: duration, dtype: float64

In [9]:
#Fraction of missing values - we are working with df now not df_1 and df_2

df_1['PUlocationID'].isnull().sum()/df_1.shape[0]

0.8303067639882438

In [10]:
df_1.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [11]:
#we will replace all Nan's in PULocationID with -1

df_1['PUlocationID'].fillna(-1,inplace=True)
df_2['PUlocationID'].fillna(-1,inplace=True)

df_1 = df_1[(df_1.duration >= 1) & (df_1.duration <= 60)]

categorical = ['PUlocationID', 'DOlocationID']
#numerical = ['duration']

df_1[categorical] = df_1[categorical].astype(str)

In [12]:
#convert the train to dictionary
train_dicts = df_1[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
X_train.shape

#column count is 525

(1109826, 525)

In [14]:
target = 'duration'
y_train = df_1[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107204271

In [15]:
#test results on validation
y_test = df_2['duration'].values

test_dicts = df_2[categorical].to_dict(orient='records')

X_test = dv.transform(test_dicts)


y_pred_test = lr.predict(X_test)

mean_squared_error(y_test, y_pred_test, squared=False)

161.1106537295235